# EMC Data

In [2]:
import os
from os.path import join

ROOT = '/home/sdc/DR_DemandForecast/emcData'
DATADIR = '/home/sdc/DR_DemandForecast/emcData/data/'

In [3]:
from dep import nemsData2 as nems
import pandas as pd
import urllib3
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
import datetime as dt
from datetime import timedelta as delta
import pytz
import time as t

## Fetch Data

### General Purpose Methods

#### Corp Data

In [ ]:
def corp_for_dpr(corp_df):
    corp_df = corp_df.copy()
    corp_df = corp_df[['Date', 'Period'] +
                      [col for col in corp_df.columns if col not in ['Date', 'Period', 'reportType', 'secondaryReserve']]]

    # Convert to int
    corp_df['Period'] = corp_df['Period'].astype(int)

    # Convert to float
    float_cols = ['Demand', 'TCL', 'USEP', 'LCP', 'Regulation', 'PrimaryReserve']
    corp_df[float_cols] = corp_df[float_cols].apply(pd.to_numeric, errors='coerce')

    # If 'secondaryReserve' and 'contingencyReserve' contain 'None', convert to float and keep NaN
    corp_df[['ContingencyReserve', 'EHEUR', 'Solar']] = corp_df[[
        'ContingencyReserve', 'EHEUR', 'Solar']].apply(pd.to_numeric, errors='coerce')
    
    return corp_df

In [ ]:
def corp_for_lar(corp_df):
    corp_df = corp_df.copy()
    corp_df = corp_df[['Date', 'Period'] +
                      [col for col in corp_df.columns if col not in ['Date', 'Period', 'reportType', 'secondaryReserve', 'Demand', 'TCL', 'USEP', 'LCP', 'EHEUR', 'Solar']]]

    # Convert 'period' to int
    corp_df['Period'] = corp_df['Period'].astype(int)

    # Convert 'regulation', 'primaryReserve', 'contingencyReserve' to float
    float_cols = ['Regulation', 'PrimaryReserve', 'ContingencyReserve']
    corp_df[float_cols] = corp_df[float_cols].apply(
        pd.to_numeric, errors='coerce')

    return corp_df

#### MCR010 Data

In [ ]:
def get_mcr010(mcr_serie):
    mcr010 = nems.getMCRReport('MCR010', mcr_serie.iloc[0])
    return mcr010

#### MCR012 Data

In [ ]:
def get_mcr012(mcrSerie):
    mcr012_df = nems.getMCRReport('MCR012', mcrSerie.iloc[0])
    return mcr012_df

#### Network Traffic Control 

In [ ]:
def wait_retry (func, targetType=pd.DataFrame):

    for i in range(10):
        try:
            data = func
            
            if not type(data) == targetType:
                print(data)
                raise Exception(f"Expected {targetType} but got {type(data)}")
            
            print('.', end='')
            t.sleep(10) # Sleep awhile if successfully fetched data
            return data
            
        except Exception as e:
            print(f"In API call, error message: {e}")
            t.sleep(30) # Sleep for a long time to wait for next try
            continue


### Source Specific Methods

#### Corp Data

In [ ]:
def fetch_corp(
    date_today: dt.date, 
    need_tomorrow_data: bool, 
    date_tomorrow: dt.date,
):
    corp_df = wait_retry(nems.getCorp(date_today.strftime(format='%d-%b-%Y')))
    corp_today = corp_for_dpr(corp_df)

    if need_tomorrow_data:
        corp_df = wait_retry(nems.getCorp(date_tomorrow.strftime(format='%d-%b-%Y')))
        corp_tomorrow = corp_for_lar(corp_df)
    else:
        print("_", end="")
        corp_tomorrow = None
        
    return corp_today, corp_tomorrow

#### MCR Data

In [ ]:
def fetch_mcr_dpr(
    date_today: dt.date, 
    period_now: int
): 
    ''' 
    ### MCR for DPR
    
    MCR001, MCR010, MCR012
    '''
 
    ''' MCR001 '''
    mcr_df = wait_retry(
        nems.getMCR001(
            date_today.strftime(format='%d-%b-%Y'), 
            'M'
        )
    )
    mcr_serie = mcr_df[
        (mcr_df['FirstDate'] == date_today) & 
        (mcr_df['FirstPeriod'] == period_now)].copy()
    
    ''' MCR010 '''
    mcr010_df = wait_retry(get_mcr010(mcr_serie))
        
    ''' MCR012 '''
    mcr012_df = wait_retry(get_mcr012(mcr_serie))
        
    return mcr010_df, mcr012_df

In [ ]:
def fetch_mcr_lar(
    date_lar: dt.date, 
    period_lar: int,
    
    loadscenario: str
): 
    '''
    ### MCR Report for LAR

    The period for LAR is 1 period later than DPR.
    
    MCR report for LAR needs a load scenario to be specified.
    '''
    
    
    ''' MCR001 '''
    mcr_df = wait_retry(
        nems.getMCR001(
            date_lar.strftime(format='%d-%b-%Y'), 
            loadscenario, 
            runType='LAR'
        )
    )
    
    mcr_serie = mcr_df[
        (mcr_df['FirstDate'] == date_lar) & 
        (mcr_df['FirstPeriod'] == period_lar)].copy()
    
    ''' MCR010 '''
    mcr010_df = wait_retry(get_mcr010(mcr_serie))
        
    ''' MCR012 '''
    mcr012_df = wait_retry(get_mcr012(mcr_serie))
        
    return mcr010_df, mcr012_df

## Join Data

### Methods

#### DPR

In [ ]:
def mcr010_for_dpr(mcr010_df):   
    mcr010_df = mcr010_df.copy() 
    mcr010_df = mcr010_df[['ForecastDate', 'ForecastPeriod', 'CUSEP', 'TransmissionLoss', 'EnergyShortfall', 'RLQ']].copy()
    mcr010_df['ForecastPeriod'] = mcr010_df['ForecastPeriod'].astype(int)
    
    return mcr010_df

In [ ]:
def mcr012_for_dpr(mcr012_df):
    
    mcr012_df = mcr012_df.copy()
    
    # Flatten MCR012
    mcr012_data = {}
    for index, row in mcr012_df.iterrows():
        mcr012_data["ForecastDate"] = [row["ForecastDate"]]
        mcr012_data["ForecastPeriod"] = [row["ForecastPeriod"]]
        mcr012_data[f"{row['AncillaryService']}_ReserveRequirement"] = [row["ReserveRequirementMW"]]
        mcr012_data[f"{row['AncillaryService']}_RegulationShortfall"] = [row["RegulationShortfallMW"]]
    
    # Create a new MCR012 DataFrame
    mcr012_df = pd.DataFrame(mcr012_data)
    
    mcr012_df.rename(
        {
            'REGULATION_ReserveRequirement': 'RegulationRequirement',
            'REGULATION_RegulationShortfall': 'RegulationShortfall',
            'PRIMARY RESERVE_ReserveRequirement': 'PrimaryReserveRequirement',
            'PRIMARY RESERVE_RegulationShortfall': 'PrimaryReserveShortfall',
            'CONTINGENCY RESERVE_ReserveRequirement': 'ContingencyReserveRequirement',
            'CONTINGENCY RESERVE_RegulationShortfall': 'ContingencyReserveShortfall'
        },
        axis=1,
        inplace=True
    )
    mcr012_df['ForecastPeriod'] = mcr012_df['ForecastPeriod'].astype(int)
    
    return mcr012_df

In [ ]:
def get_dpr(
    date_today: dt.date, 
    period_now: int, 
    corp_today: pd.DataFrame, 
    mcr010_df: pd.DataFrame, 
    mcr012_df: pd.DataFrame
):
    
    mcr010_df = mcr010_for_dpr(mcr010_df)
    mcr012_df = mcr012_for_dpr(mcr012_df)
    
    mcr_df = pd.merge(mcr010_df, mcr012_df, how='inner',
                      on=['ForecastDate', 'ForecastPeriod'])
    
    mcr_df.rename({
        'ForecastDate': 'Date',
        'ForecastPeriod': 'Period'
        }, 
        axis=1, 
        inplace=True
    )

    corp_peri_df = corp_today[
        (corp_today['Date'] == date_today) & 
        (corp_today['Period'] == period_now)].copy()
    
    dpr_df = pd.merge(corp_peri_df, mcr_df, how='inner',
                        on=['Date', 'Period'])

    float64_cols = ['Demand', 'TCL', 'USEP', 'LCP', 'Regulation',
                    'PrimaryReserve', 'ContingencyReserve', 'EHEUR', 'Solar', 'CUSEP',
                    'TransmissionLoss', 'EnergyShortfall', 'RLQ', 'RegulationRequirement',
                    'RegulationShortfall', 'PrimaryReserveRequirement',
                    'PrimaryReserveShortfall', 'ContingencyReserveRequirement',
                    'ContingencyReserveShortfall']
    dpr_df[float64_cols] = dpr_df[float64_cols].astype('float64')

    dpr_df.fillna(0, inplace=True)

    dpr_df = dpr_df[['Date', 'Period',
                    'Demand', 'TCL', 'USEP', 'CUSEP',  'LCP', 'TransmissionLoss', 'EnergyShortfall',
                    'RLQ', 'Regulation', 'RegulationRequirement', 'RegulationShortfall',
                    'PrimaryReserve', 'PrimaryReserveRequirement', 'PrimaryReserveShortfall',
                    'ContingencyReserve', 'ContingencyReserveRequirement', 
                    'ContingencyReserveShortfall', 'EHEUR', 'Solar'
                    ]]
    
    print(".", end="")
    return dpr_df

#### LAR

In [ ]:
def mcr010_for_lar(date_today, period_now, mcr010_df, load_scenario):
    mcr010_df = mcr010_df.copy() 
    mcr010_df = mcr010_df[['ForecastDate', 'ForecastPeriod', 'TotalLoad', 'TCL', 'USEP', 'CUSEP', 'LCP', 'TransmissionLoss', 'EnergyShortfall', 'RLQ', 'EHEUR', 'Solar']].copy()

    # mcr010_df['ForecastPeriod'] = mcr010_df['ForecastPeriod'].astype('int64')

    # Add current date and period back to dataframe.
    mcr010_df['Period'] = period_now
    mcr010_df['Date'] = date_today

    # Add load scenario info
    mcr010_df['LoadScenario'] = load_scenario

    mcr010_df.rename(
        {
            'TotalLoad': 'Demand'
        },
        axis=1,
        inplace=True
    )

    return mcr010_df

In [ ]:
def mcr012_for_lar(date_today, period_now, mcr012_df):
    mcr012_df = mcr012_df.copy()
    
    # Flatten MCR012
    mcr012_data = {
        'ForecastDate': [],
        'ForecastPeriod': [],
        'RegulationRequirement': [],
        'RegulationShortfall': [],
        'PrimaryReserveRequirement': [],
        'PrimaryReserveShortfall': [],
        'ContingencyReserveRequirement': [],
        'ContingencyReserveShortfall': [],
    }
    
    count = 1
    for index, row in mcr012_df.iterrows():

        if count%3 == 0:
            mcr012_data['ForecastDate'].append(row["ForecastDate"])
            mcr012_data['ForecastPeriod'].append(row["ForecastPeriod"])
            count = 1
        else: 
            count += 1

        if row['AncillaryService'] == 'REGULATION':
            mcr012_data['RegulationRequirement'].append(row["ReserveRequirementMW"])
            mcr012_data['RegulationShortfall'].append(row["RegulationShortfallMW"])
        if row['AncillaryService'] == 'PRIMARY RESERVE':
            mcr012_data['PrimaryReserveRequirement'].append(row["ReserveRequirementMW"])
            mcr012_data['PrimaryReserveShortfall'].append(row["RegulationShortfallMW"])
        if row['AncillaryService'] == 'CONTINGENCY RESERVE':
            mcr012_data['ContingencyReserveRequirement'].append(row["ReserveRequirementMW"])
            mcr012_data['ContingencyReserveShortfall'].append(row["RegulationShortfallMW"])
    

    # Create a new MCR012 DataFrame
    mcr012_df = pd.DataFrame(mcr012_data)
    mcr012_df['Date'] = date_today
    mcr012_df['Period'] = period_now

    return mcr012_df

In [ ]:
def get_lar(
    date_today: dt.date,
    date_lar: dt.date, 
    period_now: int, 
    period_lar: int,
    need_tomorrow_data: bool,
    date_tomorrow: dt.date,
    
    corp_today: pd.DataFrame,
    corp_tomorrow: pd.DataFrame, 
    mcr010_df: pd.DataFrame, 
    mcr012_df: pd.DataFrame,
    
    load_scenario: str
):
    mcr010_df = mcr010_for_lar(date_today, period_now, mcr010_df, load_scenario)
    mcr012_df = mcr012_for_lar(date_today, period_now, mcr012_df)
    
    mcr_df = pd.merge(mcr010_df, mcr012_df, how='left',
                      on=['ForecastDate', 'ForecastPeriod','Date','Period'])
    
    corp_df = corp_today[
        (corp_today['Date'] == date_lar) &
        (corp_today['Period'] >= period_lar) &
        (corp_today['Period'] < period_lar + 12)
        ].copy()
    
    if need_tomorrow_data:
        # Example 1: now is period 37, then LAR data range involves
        #   - Today: period 38 - 48, 11 periods in total.
        #   - Tomorrow: period 1, 1 period in total.
        
        # Example 2, now is period 48, then LAR data range involves
        #   - Today: No data.
        #   - Tomorrow: period 1 - 12, 12 period in total.
        
        count_period_today = 48 - period_now
        period_lar_end = 12 - count_period_today
        # print(count_period_today, count_period_tomorrow, period_lar_end)
        
        corp_df_tomorrow = corp_tomorrow[
            (corp_tomorrow['Date'] == date_tomorrow) & 
            (corp_tomorrow['Period'] <= period_lar_end)].copy()

        corp_df = pd.concat([corp_df, corp_df_tomorrow])
        
    corp_df.rename(
        {
            "Date": "ForecastDate",
            "Period": "ForecastPeriod"
        },
        axis=1,
        inplace=True
    )

    corp_df = corp_df[[
        "ForecastDate", "ForecastPeriod", 
        'Regulation', 'PrimaryReserve', 'ContingencyReserve'
        ]]

    # return corp_df

    lar_df = pd.merge(corp_df, mcr_df, how='left',
                      on=['ForecastDate', 'ForecastPeriod'])
    # return lar_df

    float64_cols = ['Demand', 'TCL', 'USEP', 'CUSEP', 'LCP', 'TransmissionLoss',
                    'EnergyShortfall', 'RLQ', 'EHEUR', 'RegulationRequirement', 'Regulation', 
                    'RegulationShortfall', 'PrimaryReserveRequirement', 'PrimaryReserve', 
                    'PrimaryReserveShortfall', 'ContingencyReserveRequirement', 
                    'ContingencyReserve', 'ContingencyReserveShortfall', 'Solar']

    lar_df[float64_cols] = lar_df[float64_cols].astype('float64')

    # lar_df['Date'] = lar_df['Date'].dt.date
    # lar_df['ForecastDate'] = lar_df['ForecastDate'].dt.date

    lar_df.fillna(0, inplace=True)

    lar_df = lar_df[['Date', 'Period', 'LoadScenario', 'ForecastDate', 'ForecastPeriod',
                    'Demand', 'TCL', 'USEP', 'CUSEP', 'LCP', 'TransmissionLoss',
                     'EnergyShortfall', 'RLQ', 'EHEUR', 'RegulationRequirement', 'Regulation', 
                     'RegulationShortfall', 'PrimaryReserveRequirement', 'PrimaryReserve', 
                     'PrimaryReserveShortfall', 'ContingencyReserveRequirement', 
                     'ContingencyReserve', 'ContingencyReserveShortfall', 'Solar'
                    ]]

    print(".", end="")
    return lar_df

## Save to DB

### Methods

#### DB Connection

In [ ]:
# Load the environment variables from the .env file
env_file = join(ROOT, '.env')
load_dotenv(env_file)

# Get the values of host, user, pswd, db, and schema from the environment variables
DBHOST = os.getenv('host')
DBUSER = os.getenv('user')
DBPSWD = os.getenv('pswd')
DBNAME = os.getenv('db')
SCHEMA = 'public'

# Use the values as needed
engine = create_engine(f"postgresql://{DBUSER}:{DBPSWD}@{DBHOST}/{DBNAME}?options=-csearch_path%3D{SCHEMA}", echo=False)
conn = engine.connect()

#### DPR to DB

In [ ]:
def db_dpr(dpr_df, schema="emcdata"):
    
    dpr_se = dpr_df.iloc[0]
    
    # check existing row
    row_exists_query = f"""
        SELECT 1 FROM {schema}."RealTimeDPR"
            WHERE "Date" = '{dpr_se['Date'].strftime(format='%Y-%m-%d')}' AND "Period" = '{dpr_se['Period']}';
        """
    row_exists = conn.execute(text(row_exists_query)).scalar()

    if row_exists:

        set_pairs = []
        d = dpr_se.to_dict()
        for k,v in d.items():
            
            if 'Date' in k:
                set_pairs.append(f""""{k}"='{v.strftime(format="%Y-%m-%d")}'""")
                continue

            set_pairs.append(f""""{k}"={float(v)}""")
        
        update_query = f'''
            UPDATE {schema}."RealTimeDPR" 
                SET {", ".join(set_pairs)}
                WHERE 
                    "Date" = '{dpr_se['Date'].strftime(format='%Y-%m-%d')}' AND 
                    "Period" = {dpr_se['Period']}
            ;
            '''
        # print(update_query)
        conn.execute(text(update_query))
        conn.commit()
        print("*", end="")
        return 0, 1
    
    else:
        cols = []
        values = []
        d = dpr_se.to_dict()
        for k,v in d.items():
            cols.append(f'''"{k}"''')
            
            if 'Date' in k:
                values.append(f"""'{v.strftime(format="%Y-%m-%d")}'""")
                continue

            values.append(f"{float(v)}")
        
        insert_query = f'''INSERT INTO {schema}."RealTimeDPR" ({", ".join(cols)}) \n VALUEs ({", ".join(values)});'''
        # print(insert_query)
        conn.execute(text(insert_query))
        conn.commit()
        print(".", end="")
        return 1, 0

#### LAR to DB

In [ ]:
def db_lar(lar_df: pd.DataFrame, schema="emcdata"):
    new = 0
    exist = 0

    for row in range(lar_df.shape[0]):
        lar_se = lar_df.iloc[row]

        # check existing row
        row_exists_query = f"""
            SELECT 1 FROM {schema}."RealTimeLAR"
                WHERE 
                    "Date"='{lar_se['Date'].strftime(format="%Y-%m-%d")}' AND 
                    "Period"={lar_se['Period']} AND 
                    "LoadScenario"='{lar_se['LoadScenario']}' AND
                    "ForecastDate"='{lar_se['ForecastDate'].strftime(format="%Y-%m-%d")}' AND 
                    "ForecastPeriod"={lar_se['ForecastPeriod']}  
            ;
            """
        # print(row_exists_query)
        row_exists = conn.execute(text(row_exists_query)).scalar()
        # print(row_exists)
        
        # continue

        if row_exists:

            set_pairs = []
            d = lar_se.to_dict()
            for k,v in d.items():
                
                if 'Date' in k:
                    set_pairs.append(f""""{k}"='{v.strftime(format="%Y-%m-%d")}'""")
                    continue

                if 'Scenario' in k:
                    set_pairs.append(f""""{k}"='{v}'""")
                    continue

                set_pairs.append(f""""{k}"={float(v)}""")
            
            update_query = f'''
                UPDATE {schema}."RealTimeLAR" 
                    SET {", ".join(set_pairs)}
                    WHERE 
                        "Date"='{lar_se['Date'].strftime(format="%Y-%m-%d")}' AND 
                        "Period"={lar_se['Period']} AND 
                        "LoadScenario"='{lar_se['LoadScenario']}' AND
                        "ForecastDate"='{lar_se['ForecastDate'].strftime(format="%Y-%m-%d")}' AND 
                        "ForecastPeriod"={lar_se['ForecastPeriod']} 
                ;
                '''
            # print(update_query)

            conn.execute(text(update_query))
            conn.commit()
            print("*", end="")
            
            exist += 1
        else:
            cols = []
            values = []
            d = lar_se.to_dict()
            for k,v in d.items():
                cols.append(f'''"{k}"''')
                
                if 'Date' in k:
                    values.append(f"""'{v.strftime(format="%Y-%m-%d")}'""")
                    continue
                
                if 'Scenario' in k:
                    values.append(f"""'{v}'""")
                    continue

                values.append(f"{float(v)}")
            
            insert_query = f'''INSERT INTO {schema}."RealTimeLAR" ({", ".join(cols)}) \n VALUEs ({", ".join(values)});'''
            # print(insert_query)

            conn.execute(text(insert_query))
            conn.commit()
            print(".", end="")

            new += 1

    return new, exist

## Main Functions

### Methods

In [ ]:
def save_to_db(
    df: pd.DataFrame,
    flag: str
):
    flag = flag.upper()
    
    if flag == 'DPR':
        new, exist = db_dpr(df, SCHEMA)
        conn.commit()
        
    elif flag == 'LAR':
        new, exist = db_lar(df, SCHEMA)
        conn.commit()
        
    return new, exist


In [ ]:
def run(
    tag: str = " ",
    date: dt.date = None, 
    period: int = None, 
    
    flag_dpr: bool = True,
    flag_lar_h: bool = True,
    flag_lar_m: bool = True,
    flag_lar_l: bool = True
):
    '''
    ### Run a data fetching and saving process

    For normal run, calling with zero params is allowed, which will use current date and time, and will go through DPR and all LAR.

    For specific purpose run, call with params.
    '''
    
    ''' Statistic '''
    exist = 0
    new = 0
    run_start = t.time()
    
    # ----------

    ''' Date and Period '''

    now = dt.datetime.now(pytz.timezone('Asia/Singapore')) # Must have a `now`. Will be used in log.

    # Use provided date
    if date:
        date_today = date
    
    # Use provided period
    if period:
        period_now = period

    # Default: use current date and time
    if not date and not period:
        date_today = now.date()
        period_now = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1
    
    # If 12 periods after current one include periods of tomorrow.
    # A.k.a Starts from period 37.
    need_tomorrow_data = period_now >= 37
    if need_tomorrow_data:
        date_tomorrow = date_today + delta(days=1)
    else:
        date_tomorrow = None

    # LAR specific start date and period
    if period_now == 48:
        date_lar = date_today + dt.timedelta(days=1)
        period_lar = 1
    else:
        date_lar = date_today
        period_lar = period_now + 1
    
    print(f"[{tag}][{date_today.strftime(format='%Y-%m-%d')} {now.time().strftime(format='%H:%M')} P-{period_now:0>2d}]", end="")
    
    
    
    ''' Corp Data '''
    
    print(" C", end="")
    # Corp data is required for both DPR and LAR
    corp_today, corp_tomorrow = fetch_corp(date_today, need_tomorrow_data, date_tomorrow)
    
    
    
    ''' DPR '''

    if flag_dpr:
        print(" D", end='')
        mcr010_df, mcr012_df = fetch_mcr_dpr(date_today, period_now)
        dpr_df = get_dpr(date_today, period_now, corp_today, mcr010_df, mcr012_df)
        new_, exist_ = save_to_db(dpr_df, 'DPR'); new += new_; exist += exist_
    else:
        print(" D SKIP", end='')
    
    ''' LAR '''
    
    if flag_lar_h:
        print(" H", end='')
        mcr010_h_df, mcr012_h_df= fetch_mcr_lar(date_lar, period_lar, 'H')
        lar_h_df = get_lar(
            date_today, date_lar, period_now, period_lar, 
            need_tomorrow_data, date_tomorrow,
            corp_today, corp_tomorrow, mcr010_h_df, mcr012_h_df, 'H'
        )
        new_, exist_ = save_to_db(lar_h_df, 'LAR'); new += new_; exist += exist_
    else:
        print(" H SKIP" + " "*11, end='')
        
        
    if flag_lar_m:
        print(" M", end='')
        mcr010_m_df, mcr012_m_df= fetch_mcr_lar(date_lar, period_lar, 'M')
        lar_m_df = get_lar(
            date_today, date_lar, period_now, period_lar, 
            need_tomorrow_data, date_tomorrow,
            corp_today, corp_tomorrow, mcr010_m_df, mcr012_m_df, 'M'
        )
        new_, exist_ = save_to_db(lar_m_df, 'LAR'); new += new_; exist += exist_
    else:
        print(" M SKIP" + " "*11, end='')
        
    if flag_lar_l:
        print(" L", end='')
        mcr010_l_df, mcr012_l_df= fetch_mcr_lar(date_lar, period_lar, 'L')
        lar_l_df = get_lar(
            date_today, date_lar, period_now, period_lar, 
            need_tomorrow_data, date_tomorrow,
            corp_today, corp_tomorrow, mcr010_l_df, mcr012_l_df, 'L'
        )
        new_, exist_ = save_to_db(lar_l_df, 'LAR'); new += new_; exist += exist_
    else:
        print(" M SKIP" + " "*11, end='')

    
    # ----------
    print(f" Updated {exist:2d} rows. Added {new:2d} rows.", end='')
    
    run_end = t.time()
    print(f" [{(run_end - run_start):.2f}s]")


In [ ]:
def safe_run(
    tag: str = " ",
    date: dt.date = None, 
    period: int = None, 
    
    flag_dpr: bool = True,
    flag_lar_h: bool = True,
    flag_lar_m: bool = True,
    flag_lar_l: bool = True
):
    try:
        run(    
            tag,
            date, 
            period, 
            
            flag_dpr,
            flag_lar_h,
            flag_lar_m,
            flag_lar_l
        )
    except Exception as e:
        print(" Aborted.") 

In [ ]:
# Query to check row counts in database
def check_avail(
    date:dt.date, 
    period:int
    ):

    ''' 
    ### Check data availability in database. 

    Return values: 
        bool, dict(str: bool)
    '''
    
    ''' DPR '''
    require_dpr = True

    # check DPR requireing row
    dpr_row_exists_query = f"""
        SELECT 1 FROM public."RealTimeDPR"
            WHERE "Date" = '{date}' AND "Period" = '{period}';
        """
    require_dpr = conn.execute(text(dpr_row_exists_query)).scalar()
    if require_dpr:
        require_dpr = False


    ''' LAR '''
    require_lar = {"H": True, "M": True, "L": True}
    # check LAR requireing row
    loadscenarios = ['H','L','M']
    
    for loadscenario in loadscenarios:
        lar_row_exists_query = f"""
            SELECT COUNT(*) FROM public."RealTimeLAR"
                WHERE 
                    "Date"='{date}' AND 
                    "Period"={period} AND 
                    "LoadScenario"='{loadscenario}' 
            ;
            """
        count_lar = int(conn.execute(text(lar_row_exists_query)).scalar())
        if count_lar == 12:
            require_lar[loadscenario] = False
    
    return require_dpr, require_lar

### Main Process

#### Normal Run

Run for current period.

In [5]:
now = dt.datetime.now(pytz.timezone('Asia/Singapore'))

In [ ]:
task_count = 1
safe_run(tag="N" + str(task_count))

#### Fill Missing A (Spec. D Spec. P)

This block can be used to check and fill missing of some specific days and periods.

In [8]:
# Set n_days to 0 to disable this block

n_days = 5 # Number of days to make up. For stability concern, maximum 5 days.
# date_start = dt.date(2024, 4, 16)
date_start = now.date() - dt.timedelta(days=11)

make_up_dates = [date_start + delta(days=i) for i in range(n_days)]
period = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1
# period = 37 # Uncomment this to overwrite with specific period.

for make_up_date in make_up_dates:
    # print(make_up_date)
    # continue

    t.sleep(30)

    require_dpr, require_lar = check_avail(make_up_date, period)
    
    if not require_dpr and not any(require_lar.values()):
        # print(period)
        continue

    task_count += 1
    safe_run(
        tag="A" + str(task_count),
        
        date=make_up_date,
        period=period,

        flag_dpr=require_dpr,
        flag_lar_h=require_lar['H'],
        flag_lar_m=require_lar['M'],
        flag_lar_l=require_lar['L'] 
    )


2024-04-15
2024-04-16
2024-04-17
2024-04-18
2024-04-19


#### Fill Missing B (Same D Prev. n P)

Fill missing data in the previous 5 period, if applicable.

In [ ]:
# Set n_period to 0 to diable this block.

n_periods = 0 # If filling specific date not enabled, maximum `n_periods` can be 8. Otherwise, 2.

period_pre = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30
periods = [period for period in range(period_pre, period_pre-n_periods, -1)]

date_yesterday =  now - dt.timedelta(days=1)

for period in periods: 
    
    if (period<1):
        period += 48
        date = date_yesterday
    else:
        date = now
        
    # print(f"{date.strftime(format='%Y-%m-%d')} P-{period:0>2d} ")
    
    require_dpr, require_lar = check_avail(date.date(), period)
    
    if not require_dpr and not any(require_lar.values()):
        # print(period)
        continue
    
    task_count += 1
    safe_run(
        tag="B" + str(task_count),
        
        date=date, 
        period=period, 
        
        flag_dpr=require_dpr,
        flag_lar_h=require_lar['H'],
        flag_lar_m=require_lar['M'],
        flag_lar_l=require_lar['L']
    )
        

#### Fill Missing C (Prev. n D Same P)

In [ ]:
# Set n_days to 0 to disable this block

n_days = 0 # Number of days to make up. For stability concern, maximum 5 days.
date_start = now.date() - dt.timedelta(days=1) # Start from yesterday.

# Use `.reverse()` here, to start checking from the earliest date.
# This can minimize the possibility of not being able to make up for some data.
make_up_dates = [date_start - dt.timedelta(days=i) for i in range(n_days)]
make_up_dates.reverse()
    
period = int(now.strftime("%H")) * 2 + int(now.strftime("%M")) // 30 + 1

for date in make_up_dates: # Set `n_days` to 0 to disable this block.

    # print(f"{date.strftime(format='%Y-%m-%d')} P-{period:0>2d} ")
    
    require_dpr, require_lar = check_avail(date, period)
    
    if not require_dpr and not any(require_lar.values()):
        # print(date)
        continue
    
    task_count += 1
    safe_run(
        tag="C" + str(task_count),
        
        date=date, 
        period=period, 
        
        flag_dpr=require_dpr,
        flag_lar_h=require_lar['H'],
        flag_lar_m=require_lar['M'],
        flag_lar_l=require_lar['L']
    )
        

## End

In [ ]:
conn.close()
print("")